In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pmdarima as pm

In [2]:
hourly_mean = pd.read_csv('../../data/Fernando/hourly_mean.csv')

In [3]:
hourly_mean_test = pd.read_csv('../../data/Fernando/hourly_mean_test.csv')

In [4]:
hourly_mean.head()

,datetime,Valor,Dia,Ano,Hora,Minuto
0,2015-03-02 00:00:00,1.000,2.0,2015.0,0.0,0.0
1,2015-03-02 01:00:00,32.565,2.0,2015.0,1.0,0.0
2,2015-03-02 02:00:00,38.347,2.0,2015.0,2.0,0.0
3,2015-03-02 03:00:00,26.616,2.0,2015.0,3.0,0.0
4,2015-03-02 04:00:00,17.202,2.0,2015.0,4.0,0.0


In [5]:
hourly_mean_test.head()

,datetime,Valor
0,2016-03-01 00:00:00,27.298
1,2016-03-01 01:00:00,176.818
2,2016-03-01 02:00:00,137.341
3,2016-03-01 03:00:00,116.532
4,2016-03-01 04:00:00,84.029


In [ ]:
model = pm.auto_arima(hourly_mean['Valor'], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=24,              
                      d=None,           # let model determine 'd'
                      seasonal=True,   
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

model.summary()

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,1)[24] intercept   : AIC=100658.798, Time=17.65 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=100923.042, Time=0.11 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=100644.148, Time=16.39 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=100662.330, Time=17.03 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=102331.497, Time=0.06 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=100831.579, Time=0.23 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=100635.771, Time=108.50 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=100588.830, Time=225.93 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=42.56 sec


In [ ]:
sns.set_context('paper', font_scale=2)

In [ ]:
model.plot_diagnostics(figsize=(20,15))
plt.show()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX 

  
model = SARIMAX(hourly_mean['Valor'],  

                # as duas tuplas do Best Model
                
                order = (2,0,0),  

                seasonal_order =(0,0,0,24)) 

  

result = model.fit() 
result.summary() 

In [ ]:
start = len(hourly_mean) 

end = len(hourly_mean) + len(hourly_mean_test) - 1

  
# Predictions for one-year against the test set 

predictions = result.predict(start, end, 

                             typ = 'levels').rename("Predictions") 

  
# plot predictions and actual values 

predictions.plot(legend = True) 

hourly_mean_test['Valor'].plot(legend = True) 

In [ ]:
from sklearn.metrics import mean_squared_error 

from statsmodels.tools.eval_measures import rmse 

  
# Calculate root mean squared error 

print(rmse(hourly_mean_test["Valor"], predictions))

  
# Calculate mean squared error 

print(mean_squared_error(hourly_mean_test["Valor"], predictions))